In [1]:
!pip install pennylane
!pip install numpy
!pip install pandas
!pip install scikit-learn

In [3]:
#Problem Statement This code demonstrates how quantum machine learning can be applied to a classical dataset (Iris dataset) using quantum kernels. The objective is to perform binary classification using a support vector machine (SVM) with quantum kernels. The code includes dataset loading, preprocessing, quantum kernel creation, and model evaluation.
# Objective
# To convert the Qiskit code into PennyLane, we aim to achieve the same functionality:
# Load and preprocess the Iris dataset.
# Use a quantum feature map as a kernel.
# Train a classical SVM classifier based on the quantum kernel and evaluate its performance.

import pennylane as qml
from pennylane import numpy as np
from sklearn import svm
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report

# Load the Iris dataset
iris = load_iris()
X = iris.data
y = iris.target

# Only take 2 classes (binary classification)
X = X[y != 2]
y = y[y != 2]

# Split dataset into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Create a quantum device with 4 qubits
n_qubits = 4
dev = qml.device("default.qubit", wires=n_qubits)

# Define the quantum feature map (circuit) as a kernel
def quantum_kernel(x1, x2):
    @qml.qnode(dev)
    def circuit(x1, x2):
        qml.templates.AngleEmbedding(x1, wires=[0, 1, 2, 3])
        qml.templates.AngleEmbedding(x2, wires=[0, 1, 2, 3])
        qml.CNOT(wires=[0, 1])
        return qml.expval(qml.PauliZ(0))
    
    return circuit(x1, x2)

# Compute the quantum kernel matrix
def kernel_matrix(X1, X2):
    kernel_mat = np.zeros((X1.shape[0], X2.shape[0]))
    for i in range(X1.shape[0]):
        for j in range(X2.shape[0]):
            kernel_mat[i, j] = quantum_kernel(X1[i], X2[j])
    return kernel_mat

# Train a classical SVM using the quantum kernel matrix
X_train_kernel = kernel_matrix(X_train, X_train)
qsvm = svm.SVC(kernel='precomputed')
qsvm.fit(X_train_kernel, y_train)

# Compute the kernel matrix for the test set
X_test_kernel = kernel_matrix(X_test, X_train)
y_pred = qsvm.predict(X_test_kernel)

# Print the classification results
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.20      0.17      0.18        12
           1       0.00      0.00      0.00         8

    accuracy                           0.10        20
   macro avg       0.10      0.08      0.09        20
weighted avg       0.12      0.10      0.11        20

